<table align="center">
  <td align="center">
    <a target="_blank" href="http://inspiredk.org">
    <img align="center" src="https://i.ibb.co/Z6HZPSbH/Inspired-K-org-Logo-No-Whitespace-Extra-Small.png">InspiredK.org Website</a>
  </td>
  
  <td align="center">
    <a target="_blank" href="https://colab.research.google.com/github/InspiredK-organization/MITintrotodeeplearning/blob/master/lab2/solutions/Lab2.1 - Handwritten Digit Classification with CNNs and TensorFlow Solution.ipynb">
    <img align="center" src="https://i.ibb.co/2P3SLwK/colab.png"/>Run in Google Colab</a>
  </td>
</table>

# Copyright Information

In [ ]:
# Copyright 2025 MIT Introduction to Deep Learning. All Rights Reserved.
#
# Licensed under the MIT License. You may not use this file except in compliance
# with the License. Use and/or modification of this code outside of MIT Introduction
# to Deep Learning must reference:
#
# © MIT Introduction to Deep Learning
# http://introtodeeplearning.com
#
# Original lab is adopted from http://introtodeeplearning.com
# Lab is edited by http://InspiredK.org

# Laboratory 2: Computer Vision

# Part 1: MNIST Digit Classification

In the first portion of this lab, we will build and train a convolutional neural network (CNN) for classification of handwritten digits from the famous [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset. We will explore several methods of optimization and training processes in order to achieve the highest accuracy of classification.

First, make sure that you change your runtime to a tensor processing unit (TPU). This can be done by navigating to `Runtime > Change runtime type > Hardware accelerator > v2-8 TPU`. The usage of a TPU is critical because there are some code incompatibilities with the free Google Colab T4 GPU, and without a TPU the code will return errors and not function correctly.

If you are unable to connect to a TPU or you reach the usage limit, a CPU can be used as the hardware accelerator. However, note that the code will take about twice as long to run, so make sure that you cannot connect to a TPU before using a CPU.

Now that we are using a TPU, let's download the course repository, install dependencies, and import the relevant packages we'll need for this lab.

In [ ]:
# Install Tensorflow 2.0.
!pip install tensorflow --quiet
import tensorflow as tf

# Install opencv-python (cv2) as a dependency for the MIT package.
!pip install opencv-python --quiet # OpenCV is a framework that supports computer vision tasks like ours.

# Download and import the MIT Introduction to Deep Learning package.
!pip install mitdeeplearning --quiet
import mitdeeplearning as mdl

# Import all remaining packages.
import matplotlib.pyplot as plt # For graphical visualizations of different statistics.
import numpy as np # For nparrays.
from tqdm import tqdm # For textual progress bars.

## 1.1 MNIST dataset

The MNIST dataset is often used in deep learning, particularly for image classification, due to its simplicity and accessiblity. It contains of 70,000 grayscale images of handwritten numbers from 0 to 9, with 60,000 training images and 10,000 testing images. Each image is 28x28, and every pixel is a value from 0 to 255 as a scale of how white it is (higher means more white, lower means more black).

Let's download and load the dataset and display a few random samples from it:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data() # Download the MNIST dataset from TensorFlow.

train_images = (np.expand_dims(train_images, axis=-1)/255.).astype(np.float32) # Convert the training images to a numerical representation in a high-dimensional space.
train_labels = (train_labels).astype(np.int64) # Convert the training images' labels (integer from 0-9) to an integer format.

# Do the same for the testing images and their labels.
test_images = (np.expand_dims(test_images, axis=-1)/255.).astype(np.float32)
test_labels = (test_labels).astype(np.int64)

Let's visualize what some of these images and their corresponding training labels look like.

In [ ]:
# Display some example images and labels from our training subset.
plt.figure(figsize=(10,10)) # Create the figure to display the examples.
random_inds = np.random.choice(60000,36) # Choose 36 random indices to select images and labels.
for i in range(36): # For each index, display the image with its respective label.
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image_ind = random_inds[i]
    plt.imshow(np.squeeze(train_images[image_ind]), cmap=plt.cm.binary)
    plt.xlabel(train_labels[image_ind])

## 1.2 Neural Network for Handwritten Digit Classification

We'll first build a simple neural network consisting of two fully connected layers and apply this to the digit classification task. Our network will ultimately output a probability distribution over the 10 digit classes (0-9). This first architecture we will be building is depicted below:

![alt_text](https://raw.githubusercontent.com/MITDeepLearning/introtodeeplearning/master/lab2/img/mnist_2layers_arch.png "CNN Architecture for MNIST Classification")


### Fully connected neural network architecture
To define the architecture of this first fully connected neural network, we'll once again use the Keras API and define the model using the [`Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential) class. Note how we first use a [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten) layer, which flattens the input so that it can be fed into the model.

In this next block, you'll define the fully connected layers of this simple work.

In [ ]:
def build_model():
  model = tf.keras.Sequential([
      # Create a Flatten layer that turns our 2-dimensional image into a 1-dimensional vector.
      tf.keras.layers.Flatten(),

      # '''TODO: Define the activation function for the first fully connected (Dense) layer.'''
      tf.keras.layers.Dense(128, activation=tf.nn.relu), # Create a dense layer with 128 neurons and a ReLU non-linearity.
      # tf.keras.layers.Dense(128, activation= '''TODO'''),

      # '''TODO: Define the second Dense layer to output the classification probabilities.'''
      tf.keras.layers.Dense(10, activation=tf.nn.softmax) # Create a final dense layer with 10 neurons to output the classification probabilities.
      # [TODO Dense layer to output classification probabilities]

  ])
  return model

model = build_model()

As we progress through this next portion, you may find that you'll want to make changes to the architecture defined above. **Note that in order to try new training parameters later on, you'll need to re-run the above cell to re-initialize the model. If the model is not re-initialized, the updated training will add on to the previous one.**

Let's take a step back and think about the network we've just created. The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a 2d-array (28 x 28 pixels), to a 1d-array of 28 * 28 = 784 pixels. You can think of this layer as unstacking rows of pixels in the image and lining them up. There are no learned parameters in this layer; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are fully-connected neural layers. The first `Dense` layer has 128 nodes (or neurons). The second (and last) layer (which you've defined!) should return an array of probability scores that sum to 1. Each node contains a score that indicates the probability that the current image belongs to one of the handwritten digit classes.

That defines our fully connected model!



### Compile the model

Before training the model, we need to define a few more settings. These are added during the model's [`compile`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#compile) step:

* *Loss function* — This defines how we measure how accurate the model is during training. As was covered in lecture, during training we want to minimize this function, which will "steer" the model in the right direction.
* *Optimizer* — This defines how the model is updated based on the data it sees and its loss function.
* *Metrics* — Here we can define metrics used to monitor the training and testing steps. In this example, we'll look at the *accuracy*, the fraction of the images that are correctly classified.

We'll start out by using a stochastic gradient descent (SGD) optimizer initialized with a learning rate of 0.1. Since we are performing a categorical classification task, we'll want to use the [cross entropy loss](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/sparse_categorical_crossentropy).

You'll want to experiment with both the choice of optimizer and learning rate and evaluate how these affect the accuracy of the trained model.

In [ ]:
'''TODO: Experiment with different optimizers and learning rates. How do these affect
    the accuracy of the trained model? Which optimizers and learning rates yield the best performance?'''
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), # Our model will use the Adam optimizer and a learning rate of 1e-3 (0.001).
              loss='sparse_categorical_crossentropy', # Cross entropy losses are well-suited for probability tasks, and this type is very efficient and simple.
              metrics=['accuracy']) # We will need to log the accuracy of our model in addition to the loss to track its performance overall.

### Train the model

We're now ready to train our model, which will involve feeding the training data (`train_images` and `train_labels`) into the model, and then asking it to learn the associations between images and labels. We'll also need to define the batch size and the number of epochs, or iterations over the MNIST dataset, to use during training.

In Lab 1, we saw how we can use `GradientTape` to optimize losses and train models with stochastic gradient descent. After defining the model settings in the `compile` step, we can also accomplish training by calling the [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit) method on an instance of the `Model` class. We will use this to train our fully connected model


In [ ]:
# Define the batch size and the number of epochs to use during training.
BATCH_SIZE = 32 # Number of training images and labels fed through the model at a time for the entire training dataset.
EPOCHS = 10 # Number of times the training dataset is looped through.

# The .fit function is a easy way to quickly train our model, and we will later explore more complex ways of training our model.
model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS) # Remember to re-run the previous cell to re-initialize the model when experimenting.
# If the model is not re-initialized, the updated training will add on to the previous one.

As the model trains, the loss and accuracy metrics are displayed. With five epochs and a learning rate of 0.01, this fully connected model should achieve an accuracy of approximately 0.97 (or 97%) on the training data.

### Evaluate accuracy on the test dataset

Now that we've trained the model, we can ask it to make predictions about a test set that it hasn't seen before. In this example, the `test_images` array comprises our test dataset. To evaluate accuracy, we can check to see if the model's predictions match the labels from the `test_labels` array.

Use the [`evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate) method to evaluate the model on the test dataset!

In [ ]:
'''TODO: Use the evaluate method to test the model!'''
test_loss, test_acc = model.evaluate(test_images, test_labels) # Find the loss and accuracy of our model on the testing dataset after training.
# test_loss, test_acc = # TODO

# Let's examine these metrics.
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

You may observe that the accuracy on the test dataset is a little lower than the accuracy on the training dataset. This gap between training accuracy and test accuracy is an example of *overfitting*, when a machine learning model performs worse on new data than on its training data.

What is the highest accuracy you can achieve with this first fully connected model? Since the handwritten digit classification task is pretty straightforward, you may be wondering how we can do better...

![We need to go deeper...](https://i.kym-cdn.com/photos/images/newsfeed/000/534/153/f87.jpg)

## 1.3 Convolutional Neural Network (CNN) for handwritten digit classification

As we saw in lecture, convolutional neural networks (CNNs) are particularly well-suited for a variety of tasks in computer vision, and have achieved near-perfect accuracies on the MNIST dataset. We will now build a CNN composed of two convolutional layers and pooling layers, followed by two fully connected layers, and ultimately output a probability distribution over the 10 digit classes (0-9). The CNN we will be building is depicted below:

![alt_text](https://raw.githubusercontent.com/MITDeepLearning/introtodeeplearning/master/lab2/img/convnet_fig.png "CNN Architecture for MNIST Classification")

### Define the CNN model

We'll use the same training and test datasets as before, and proceed similarly as our fully connected network to define and train our new CNN model. To do this we will explore two layers we have not encountered before: you can use  [`keras.layers.Conv2D` ](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) to define convolutional layers and [`keras.layers.MaxPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) to define the pooling layers. Use the parameters shown in the network architecture above to define these layers and build the CNN model.

In [ ]:
def build_cnn_model():
    cnn_model = tf.keras.Sequential([

        # TODO: Define the first convolutional layer.
        # We will use 24 convolutional filters for every 3 by 3 part of the input.
        tf.keras.layers.Conv2D(filters=24, kernel_size=(3,3), activation=tf.nn.relu),
        # tf.keras.layers.Conv2D('''TODO''')

        # TODO: Define the first max pooling layer.
        # We will take the maximum value for every 2 by 2 part of the convolution's output.
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # tf.keras.layers.MaxPool2D('''TODO''')

        # TODO: Define the second convolutional layer.
        # We will use 36 convolutional filters for every 3 by 3 part of the pooling output for a second convolution.
        tf.keras.layers.Conv2D(filters=36, kernel_size=(3,3), activation=tf.nn.relu),
        # tf.keras.layers.Conv2D('''TODO''')

        # TODO: Define the second max pooling layer.
        # We will again take the maximum value for every 2 by 2 part of the second convolution's output.
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # tf.keras.layers.MaxPool2D('''TODO''')

        tf.keras.layers.Flatten(), # We now convert our 2-dimensional output to a 1-dimensional vector.
        tf.keras.layers.Dense(128, activation=tf.nn.relu), # Like with our first model, we put this vector through a dense layer.

        # TODO: Define the last Dense layer to output the classification.
        # probabilities. Pay attention to the activation needed a probability output.
        tf.keras.layers.Dense(10, activation=tf.nn.softmax) # Finally, we output our classification probabilities for 0-9.
        # [TODO Dense layer to output classification probabilities]
    ])

    return cnn_model

cnn_model = build_cnn_model()
cnn_model.predict(train_images[[0]]) # Initialize the model by passing some data through it.
print(cnn_model.summary()) # Print the model summary for all of our model's layers.

### Train and test the CNN model

Now, as before, we can define the loss function, optimizer, and metrics through the `compile` method. Compile the CNN model with an optimizer and learning rate of choice:

In [ ]:
'''TODO: Define the compile operation with your optimizer and learning rate of choice.'''
# We will initialize the model like before.
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3), # Our model will use the Adam optimizer, now with a learning rate of 5e-3 (0.005).
              loss='sparse_categorical_crossentropy', # Sparse categorical cross-entropy loss for probability task usage and efficiency.
              metrics=['accuracy']) # Logging the accuracy along with the loss.
# cnn_model.compile(optimizer='''TODO''', loss='''TODO''', metrics=['accuracy']) # TODO

As was the case with the fully connected model, we can train our CNN using the `fit` method via the Keras API.

In [ ]:
'''TODO: Use model.fit to train the CNN model, with a new batch size and number of epochs.'''
BATCH_SIZE = 32 # Number of training images and labels fed through the model at a time for the entire training dataset.
EPOCHS = 5 # Number of times the training dataset is looped through.

# Like before, we use the .fit function to train our model.
cnn_model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS) # Also like before, re-run the previous cell to re-initialize the model when experimenting.
# cnn_model.fit('''TODO''')

Great! Now that we've trained the model, let's evaluate it on the test dataset using the [`evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate) method:

In [ ]:
'''TODO: Use the evaluate method to test the model!'''
test_loss, test_acc = cnn_model.evaluate(test_images, test_labels) # Find the loss and accuracy of our model on the testing dataset.
# test_loss, test_acc = # TODO

# Let's examine these metrics.
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

What is the highest accuracy you're able to achieve using the CNN model, and how does the accuracy of the CNN model compare to the accuracy of the simple fully connected network? What optimizers and learning rates seem to be optimal for training the CNN model?

Feel free to click the Comet links to investigate the training/accuracy curves for your model.

### Make predictions with the CNN model

With the model trained, we can use it to make predictions about some images. The [`predict`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#predict) function call generates the output predictions given a set of input samples.


In [ ]:
predictions = cnn_model.predict(test_images) # Get the model's predictions for labels on our testing set.

With this function call, the model has predicted the label for each image in the testing set. Let's take a look at the prediction for the first image in the test dataset:

In [ ]:
predictions[0] # Examine the prediction probability distribution for our first testing image.

As you can see, a prediction is an array of 10 numbers. Recall that the output of our model is a probability distribution over the 10 digit classes. Thus, these numbers describe the model's "confidence" that the image corresponds to each of the 10 different digits.

Let's look at the digit that has the highest confidence for the first image in the test dataset:

In [ ]:
'''TODO: identify the digit with the highest confidence prediction for the first image in the test dataset. '''
prediction = np.argmax(predictions[0]) # Find the predicted digit that has the highest probability.
# prediction = # TODO
print(prediction)

So, the model is most confident that this image is a 7. We can check the test label (the true identity of the digit) to see if this prediction is correct:

In [ ]:
print("Label of this digit is:", test_labels[0]) # Check to see if our model correctly predicted the digit correctly.
plt.imshow(test_images[0,:,:,0], cmap=plt.cm.binary) # Display the image for our example prediction.

It is! Let's visualize the classification results on the MNIST dataset. We will plot images from the test dataset along with their predicted label, as well as a histogram that provides the prediction probabilities for each of the digits:

In [ ]:
#@title Change the slider to look at the model's predictions! { run: "auto" }

# Examine the test images, their labels and their predictions for the first 500
image_index = 0 #@param {type:"slider", min:0, max:500, step:1}
                                                #  ^^^ Change this for more examples.

plt.subplot(1,2,1)
mdl.lab2.plot_image_prediction(image_index, predictions, test_labels, test_images)
plt.subplot(1,2,2)
mdl.lab2.plot_value_prediction(image_index, predictions, test_labels)

We can also plot several images along with their predictions, where correct prediction labels are blue and incorrect prediction labels are grey. The number gives the percent confidence (out of 100) for the predicted label. Note the model can be very confident in an incorrect prediction!

In [ ]:
num_rows = 5 # Number of rows in the graph.
num_cols = 4 # Number of columns in the graph.
num_images = num_rows*num_cols # Number of images in the graph.
plt.figure(figsize=(2*2*num_cols, 2*num_rows))

for i in range(num_images): # For each image, display its label and its prediction.
  # Red means the prediction is wrong, blue means the prediction is right.
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  mdl.lab2.plot_image_prediction(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  mdl.lab2.plot_value_prediction(i, predictions, test_labels)

## 1.4 Training the model 2.0

Earlier in the lab, we used the [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit) function call to train the model. This function is quite high-level and intuitive, which is really useful for simpler models. As you may be able to tell, this function abstracts away many details in the training call, and we have less control over training model, which could be useful in other contexts.

As an alternative to this, we can use the [`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape) class to record differentiation operations during training, and then call the [`tf.GradientTape.gradient`](https://www.tensorflow.org/api_docs/python/tf/GradientTape#gradient) function to actually compute the gradients. You may recall seeing this in Lab 1 Parts 1 and 2, but let's take another look at this here.

We'll use this framework to train our `cnn_model` using stochastic gradient descent.

In [ ]:
cnn_model = build_cnn_model() # Rebuild the CNN model to forget any previous training.

# Try experimenting with these to see what gets the lowest loss.
BATCH_SIZE = 32 # Define our batch size for the training process.
LEARNING_RATE = 5e-3 # Define our learning rate for the training process.

loss_history = mdl.util.LossHistory() # Create a list to track our loss during the training process.
plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Loss') # Create a graph to plot the loss during the training process.

optimizer = tf.keras.optimizers.Adagrad(learning_rate=LEARNING_RATE) # Define our optimizer with the learning rate.

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # Clear any previous progress bars if they exist.

for idx in tqdm(range(0, train_images.shape[0], BATCH_SIZE)):
  # First, get a batch of training data and turn the images into numerical representations.
  (images, labels) = (train_images[idx:idx+BATCH_SIZE], train_labels[idx:idx+BATCH_SIZE])
  images = tf.convert_to_tensor(images, dtype=tf.float32)

  # Use GradientTape to compute the gradient and backpropogate to decrease loss.
  with tf.GradientTape() as tape:
    #'''TODO: feed the images into the model and obtain the predictions.'''
    logits = cnn_model(images) # Get the classification predictions for our inputs.
    # logits = # TODO

    #'''TODO: compute the categorical cross entropy loss.
    loss_value = tf.keras.backend.sparse_categorical_crossentropy(labels, logits) # Get the loss for our predictions vs the actual labels.
    # loss_value = tf.keras.backend.sparse_categorical_crossentropy('''TODO''', '''TODO''') # TODO

  loss_history.append(loss_value.numpy().mean()) # Add the loss to the loss_history list.
  plotter.plot(loss_history.get()) # Graph the new loss_history list for progress tracking.

  '''TODO: Use the tape to compute the gradient against all parameters in the CNN model.
      Use cnn_model.trainable_variables to access these parameters.'''
  grads = tape.gradient(loss_value, cnn_model.trainable_variables) # Compute the gradient to perform backpropagation.
  # grads = # TODO
  optimizer.apply_gradients(zip(grads, cnn_model.trainable_variables)) # Use the gradient with the optimizer to change our weights and lower our model's loss.
print('Final loss after training:', loss_value.numpy().mean())


## 1.5 Conclusion
In this part of the lab, we had the chance to try out different MNIST classifiers with different architectures like fully-connected layers and CNNs. Then, we experimented with how different hyperparameters like the learning rate affect accuracy. In the next part of the lab, we will explore the application of CNNs for facial detection and examine some of the drawbacks of AI systems in real world applications like the problem of bias.